# **Dependências do Projeto**

In [1]:
%pip install -U selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install torch torchvision torchaudio
#Instalando o pytorch

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
#Caso não consiga pydirectinput
%pip install pyautogui

Note: you may need to restart the kernel to use updated packages.


In [5]:
#Biblioteca de aprendizado por reforço
%pip install stable-baselines3[extra] protobuf==3.20.*

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install mss pydirectinput pytesseract

Note: you may need to restart the kernel to use updated packages.


# **Imports**

In [63]:
#Para captura de tela
from mss import mss
#import pyautogui #para comandos de teclado
import cv2 
#Tratamentos dos frames
import numpy as np 
import pyautogui
#Framework intermediário paara trabalhar com as imagens
#import pytesseract #para extrair o game over da imagem
from matplotlib import pyplot as plt #Visualizando resultados
import time
from PIL import Image
#Coisas para construir o ambinete
from gym import Env
from gym.spaces import Box, Discrete, Dict
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from io import BytesIO
from PIL import Image
from selenium.common.exceptions import WebDriverException
import base64
from io import BytesIO


In [64]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# **Construindo o Ambiente**

In [65]:
from collections import deque

In [66]:
#Classe para definir o ambiente
class WebGame(Env):
    #ESSAS FUNÇÃO SÃO NECESSÁRIAS POR CONTA DO GYM
    #Função responsavel pela inicialização do ambiente;
    #Portanto onde ficam todas as variaveis do ambiente, configuração das ações e da observação 
    def __init__(self):
        super().__init__()

        #Configuração do tempo
        self.print_time = .005
        self.wait_time = .015
        
        #Configuração do espaço
        self.observation_space = Box(low=0, high=255, shape=(1,83,200), dtype=np.uint8)
        #self.observation_space = Dict({
        #    "print":Box(low=0, high=255, shape=(1,83,200), dtype=np.uint8),
        #    "print2":Box(low=0, high=255, shape=(1,83,200), dtype=np.uint8),
        #    "print3":Box(low=0, high=255, shape=(1,83,200), dtype=np.uint8),
            #"print4":Box(low=0, high=255, shape=(1,83,200), dtype=np.uint8),
        #    "score":Box(low=0, high=10000, shape=(1,), dtype=np.uint8),
        #                              })
        self.action_space = Discrete(3)

        #Variavei para a captura de tela
        self.screen_cap = mss() #Lib de captura de tela
        self.game_location = {"top": 300, "left": 0, "width": 650, "height": 500} #Área do monitor com 
        self.game_done = {"top": 40, "left": 0, "width": 800, "height": 640}

        self.chromedriver_path = "./chromedriver.exe"
        _chrome_options = webdriver.ChromeOptions()
        _chrome_options.add_argument("--mute-audio")
        _chrome_options.add_argument("--disable-gpu")
        self._driver = webdriver.Chrome(executable_path=self.chromedriver_path,options=_chrome_options)

        self.first_image = np.zeros(5)
        self.last_image = np.zeros((500,650,3))
        
        #Dicionário de ações
        self.action_dict = {
            0:Keys.ARROW_DOWN,
            1:Keys.ARROW_UP,
            2:'nothing'
        }

    #Função responsavel por passar as ações pro jogo, para fazer algo pro jogo
    def step(self, action):
        # Se ação escolhida for um dos botões, realizar a ação.
        if action != 2:
            self._driver.find_elements(By.TAG_NAME, "body")[0].send_keys(self.action_dict[int(action)])
        
        #Checa a próxima ação
        observation = self.get_observation()
        #Todas as ações tem que verificar se a ação acabou
        done, done_cap = self.get_done() 
        
        #Pegando o score do jogo
        score = self.get_score()
        #Ganhamos 1 ponto por cada frame que estamos vivos.
        reward = 1
        #É um dicionário de informações que retornam através do que a gente precisa
        info = {
            'score':score
        }

        time.sleep(self.wait_time)
        
        return observation, reward, done, info
        

    #Função para a visualização do jogo:
    def render(self, mode: str='human'):
        img = cv2.cvtColor(self._get_image(), cv2.COLOR_BGR2RGB)
        if mode == 'rgb_array':
            return img
        elif mode == 'human':
            from gym.envs.classic_control import rendering
            if self.viewer is None:
                self.viewer = rendering.SimpleImageViewer()
            self.viewer.imshow(img)
            return self.viewer.isopen

        # cv2.imshow('Game', np.array(self.cap.grab(self.game_location))[:,:,:3])
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     self.close()

    #Função responsavel por recomeçar o jogo
    def reset(self):
        #Try catch precisa existir pq toda a vez q o selenium detecta que ele viu o chrome dino,
        #ele dispara um erro que esta sem internet.
        try:
            self._driver.get('chrome://dino')
            
        except WebDriverException:
            pass
        WebDriverWait(self._driver, 10).until(
            EC.presence_of_element_located((
                By.CLASS_NAME, 
                "runner-canvas"
            ))
        )
        self._driver.implicitly_wait(0.5)
        self._driver.find_elements(By.TAG_NAME, "body")[0].send_keys(Keys.ARROW_UP)
        

        return self.get_observation()

    #FUNÇÕES CUSTOM
    #Fecha a parte de visualização
    def close(self):
        cv2.destroyAllWindows()
    
    def get_score(self):
        score_array = self._driver.execute_script("return Runner.instance_.distanceMeter.digits")
        score = ''.join(score_array)
        #shape = np.zeros(1)
        #shape[0] = int(score)
        return int(score)
    
    def get_score_shape(self):
        shape = np.zeros(1)
        shape[0] = int(self.get_score())
        return shape
    
    def get_img(self):
        LEADING_TEXT = "data:image/png;base64,"
        canvas = self._driver.execute_script("return document.querySelector('canvas.runner-canvas').toDataURL()")
        img = canvas[len(LEADING_TEXT):]
        img_data = np.array(Image.open(BytesIO(base64.b64decode(img))))
        return img_data 
        # img = self._driver.execute_script()
    
    #Função responsavel por pegar a imagem do jogo e tratar da forma necessária.
    def get_print(self): 

        #img = numpy.array(self.screen_cap.grab(game_location))[:,:,3].astype(np.uint8)
        raw = np.array(self.get_img())[:,:,:3].astype(np.uint8)
        img = raw[:200,:400]
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (200,83))
        channel = np.reshape(resized, (1,83,200))
        
        self.first_image = self.last_image
        self.last_image = raw
        
        return channel
    
    #Função responsavel por pegar a imagem do jogo e tratar da forma necessária.
    def get_observation(self): 
        #print1 = self.get_print()
        #self._driver.implicitly_wait(self.print_time)
        #print2 = self.get_print()
        #self._driver.implicitly_wait(self.print_time)
        #return {"print":print1, "print2":print2, "print3":self.get_print(), "score":self.get_score_shape() }
        return self.get_print()


    #Função para pegar o texto de fim de jogo:
    def get_done(self):
        #Se as duas imagens iniciais forem iguais acabou jogo
        done = False
        #Colando a primeira condição devido ao fator de que as formas diferentes estavam dando problema na comparação devido ao np.all
        
        if self.first_image.shape != self.last_image.shape:
            return done, self.last_image

        if np.all(np.equal(self.first_image, self.last_image)) and not self._driver.execute_script("return Runner.instance_.playing"):
            done = True
        
        return done, self.last_image

# Treinamento

In [67]:
# Import os for file path management
import os 
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback, CheckpointCallback, CallbackList
# Verificando se o ambiente é válido para fazer as coisa
from stable_baselines3.common import env_checker

from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, SubprocVecEnv

In [68]:
from stable_baselines3 import DQN, PPO, DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack,SubprocVecEnv
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.logger import Figure, configure

In [69]:
#env_lambda = lambda: WebGame()
#env2 = SubprocVecEnv([env_lambda for i in range(4)])
env2 = WebGame()

C:\Users\Dan\AppData\Local\Temp\ipykernel_11700\830487098.py:33: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self._driver = webdriver.Chrome(executable_path=self.chromedriver_path,options=_chrome_options)


In [70]:
env2.reset()

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=uint8)

In [71]:
#Criandos as pastas para que funcione.
CHECKPOINT_DIR = './checkpoints/'
LOG_DIR = './logs/'

In [72]:
checkpoint_callback = CheckpointCallback(
  save_freq=500,
  save_path=CHECKPOINT_DIR,
  name_prefix="dqn_single_single_model",
  save_replay_buffer=True,
  save_vecnormalize=True,
    verbose=2
)
eval_callback = EvalCallback(env2, best_model_save_path=CHECKPOINT_DIR,
                             log_path=LOG_DIR, eval_freq=500,
                             deterministic=True, render=False)
callback = CallbackList([checkpoint_callback, eval_callback])

In [73]:
def get_model():
    #model = PPO('MultiInputPolicy', env2, tensorboard_log=LOG_DIR, learning_rate=0.005, verbose=2, policy_kwargs=dict(normalize_images=False))
    model = DQN('CnnPolicy', env2, tensorboard_log=LOG_DIR, verbose=2, buffer_size=12000, learning_rate=0.0006, 
        learning_starts=500, exploration_fraction=0.4, exploration_initial_eps=0.9, exploration_final_eps=0.05,
                policy_kwargs=dict(normalize_images=False))
    return model

In [74]:
model = get_model()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [75]:
#Começa o treinamento
model.learn(total_timesteps=45000, callback=callback)

Logging to ./logs/DQN_23
Saving model checkpoint to ./checkpoints/dqn_single_single_model_500_steps.zip
Saving model replay buffer checkpoint to ./checkpoints/dqn_single_single_model_replay_buffer_500_steps.pkl
Eval num_timesteps=500, episode_reward=159.00 +/- 2.45
Episode length: 159.00 +/- 2.45
----------------------------------
| eval/               |          |
|    mean_ep_length   | 159      |
|    mean_reward      | 159      |
| rollout/            |          |
|    exploration_rate | 0.876    |
| time/               |          |
|    total_timesteps  | 500      |
----------------------------------
New best mean reward!
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 147      |
|    ep_rew_mean      | 147      |
|    exploration_rate | 0.872    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 10       |
|    time_elapsed     | 56       |
|    total_timesteps  | 587      |
| train/       

Saving model checkpoint to ./checkpoints/dqn_single_single_model_3500_steps.zip
Saving model replay buffer checkpoint to ./checkpoints/dqn_single_single_model_replay_buffer_3500_steps.pkl
Eval num_timesteps=3500, episode_reward=149.00 +/- 12.21
Episode length: 149.00 +/- 12.21
----------------------------------
| eval/               |          |
|    mean_ep_length   | 149      |
|    mean_reward      | 149      |
| rollout/            |          |
|    exploration_rate | 0.735    |
| time/               |          |
|    total_timesteps  | 3500     |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0178   |
|    n_updates        | 749      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 107      |
|    ep_rew_mean      | 107      |
|    exploration_rate | 0.718    |
| time/               |          |
|    episodes         | 36       |
|    fps              

Eval num_timesteps=6500, episode_reward=134.60 +/- 12.01
Episode length: 134.60 +/- 12.01
----------------------------------
| eval/               |          |
|    mean_ep_length   | 135      |
|    mean_reward      | 135      |
| rollout/            |          |
|    exploration_rate | 0.593    |
| time/               |          |
|    total_timesteps  | 6500     |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0156   |
|    n_updates        | 1499     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 106      |
|    ep_rew_mean      | 106      |
|    exploration_rate | 0.581    |
| time/               |          |
|    episodes         | 64       |
|    fps              | 8        |
|    time_elapsed     | 811      |
|    total_timesteps  | 6756     |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0123   |


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 105      |
|    ep_rew_mean      | 105      |
|    exploration_rate | 0.443    |
| time/               |          |
|    episodes         | 92       |
|    fps              | 8        |
|    time_elapsed     | 1185     |
|    total_timesteps  | 9674     |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.017    |
|    n_updates        | 2293     |
----------------------------------
Saving model checkpoint to ./checkpoints/dqn_single_single_model_10000_steps.zip
Saving model replay buffer checkpoint to ./checkpoints/dqn_single_single_model_replay_buffer_10000_steps.pkl
Eval num_timesteps=10000, episode_reward=128.60 +/- 6.47
Episode length: 128.60 +/- 6.47
----------------------------------
| eval/               |          |
|    mean_ep_length   | 129      |
|    mean_reward      | 129      |
| rollout/            |          |
|    exploration_rate

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 98.8     |
|    ep_rew_mean      | 98.8     |
|    exploration_rate | 0.306    |
| time/               |          |
|    episodes         | 124      |
|    fps              | 8        |
|    time_elapsed     | 1568     |
|    total_timesteps  | 12574    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0241   |
|    n_updates        | 3018     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 97.8     |
|    ep_rew_mean      | 97.8     |
|    exploration_rate | 0.292    |
| time/               |          |
|    episodes         | 128      |
|    fps              | 8        |
|    time_elapsed     | 1590     |
|    total_timesteps  | 12868    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0154   |
|    n_updates      

...

Saving model checkpoint to ./checkpoints/dqn_single_single_model_30500_steps.zip
Saving model replay buffer checkpoint to ./checkpoints/dqn_single_single_model_replay_buffer_30500_steps.pkl
Eval num_timesteps=30500, episode_reward=191.40 +/- 41.11
Episode length: 191.40 +/- 41.11
----------------------------------
| eval/               |          |
|    mean_ep_length   | 191      |
|    mean_reward      | 191      |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 30500    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0434   |
|    n_updates        | 7499     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 101      |
|    ep_rew_mean      | 101      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 316      |
|    fps           

New best mean reward!
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 105      |
|    ep_rew_mean      | 105      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 340      |
|    fps              | 7        |
|    time_elapsed     | 4426     |
|    total_timesteps  | 33675    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.00496  |
|    n_updates        | 8293     |
----------------------------------
Saving model checkpoint to ./checkpoints/dqn_single_single_model_34000_steps.zip
Saving model replay buffer checkpoint to ./checkpoints/dqn_single_single_model_replay_buffer_34000_steps.pkl
Eval num_timesteps=34000, episode_reward=157.00 +/- 30.23
Episode length: 157.00 +/- 30.23
----------------------------------
| eval/               |          |
|    mean_ep_length   | 157      |
|    mean_reward      | 157      |
| rollout/            |         

Saving model checkpoint to ./checkpoints/dqn_single_single_model_37000_steps.zip
Saving model replay buffer checkpoint to ./checkpoints/dqn_single_single_model_replay_buffer_37000_steps.pkl
Eval num_timesteps=37000, episode_reward=127.20 +/- 13.67
Episode length: 127.20 +/- 13.67
----------------------------------
| eval/               |          |
|    mean_ep_length   | 127      |
|    mean_reward      | 127      |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 37000    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0154   |
|    n_updates        | 9124     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 108      |
|    ep_rew_mean      | 108      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 372      |
|    fps           

Eval num_timesteps=40000, episode_reward=124.60 +/- 22.66
Episode length: 124.60 +/- 22.66
----------------------------------
| eval/               |          |
|    mean_ep_length   | 125      |
|    mean_reward      | 125      |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 40000    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0091   |
|    n_updates        | 9874     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 112      |
|    ep_rew_mean      | 112      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 400      |
|    fps              | 7        |
|    time_elapsed     | 5331     |
|    total_timesteps  | 40406    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0205   |

Saving model replay buffer checkpoint to ./checkpoints/dqn_single_single_model_replay_buffer_43500_steps.pkl
Eval num_timesteps=43500, episode_reward=271.00 +/- 33.09
Episode length: 271.00 +/- 33.09
----------------------------------
| eval/               |          |
|    mean_ep_length   | 271      |
|    mean_reward      | 271      |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 43500    |
| train/              |          |
|    learning_rate    | 0.0006   |
|    loss             | 0.0219   |
|    n_updates        | 10749    |
----------------------------------
New best mean reward!
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 117      |
|    ep_rew_mean      | 117      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 424      |
|    fps              | 7        |
|    time_elapsed     | 5835     |
|    tot

# Teste

In [81]:
test_env = WebGame()
model = DQN('CnnPolicy', test_env, tensorboard_log=LOG_DIR, verbose=2, buffer_size=12000, 
        learning_starts=500, policy_kwargs=dict(normalize_images=False))
model.load('checkpoints/best_model') 

for episode in range(5): 
    obs = test_env.reset()
    done = False
    total_reward = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = test_env.step(action)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))

    

C:\Users\Dan\AppData\Local\Temp\ipykernel_11700\830487098.py:33: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self._driver = webdriver.Chrome(executable_path=self.chromedriver_path,options=_chrome_options)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Total Reward for episode 0 is 212
Total Reward for episode 1 is 161
Total Reward for episode 2 is 163
Total Reward for episode 3 is 163
Total Reward for episode 4 is 159


In [82]:
%tensorboard --logdir ./logs/DQN_23/ --host localhost